# LIDC IDRI 2D SEGMENTATION

## Import Libraries

In [1]:
!pip list

Package                            Version
---------------------------------- --------------------
absl-py                            1.1.0
alabaster                          0.7.12
albumentations                     1.2.1
anaconda-client                    1.9.0
anaconda-navigator                 2.1.1
anaconda-project                   0.10.1
anyio                              3.6.1
appdirs                            1.4.4
argh                               0.26.2
argon2-cffi                        20.1.0
arrow                              0.13.1
asn1crypto                         1.4.0
astroid                            2.6.6
astropy                            4.3.1
async-generator                    1.10
atomicwrites                       1.4.0
attrs                              21.2.0
autopep8                           1.5.7
Babel                              2.9.1
backcall                           0.2.0
backports.functools-lru-cache      1.6.4
backports.shutil-get-terminal-size 

In [2]:
import pandas as pd
import argparse
import os
from collections import OrderedDict
from glob import glob
import yaml
import numpy as np

import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms.functional as TF
from torchvision import transforms
import torchsummary as summary

import albumentations as albu
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tqdm import tqdm

from Unet.unet_model import UNet
from UnetNested.Nested_Unet import NestedUNet

## Define Parameters

In [3]:
name = "UNet"           #default = "UNet"; can be NestedUNet
epochs = 100            #default = 400
batch_size = 12         #default = 12
early_stopping = 50     #default = 50
num_workers = 8         #default = 8
optimizer = 'Adam'      #default = 'Adam'; can be SGD
lr = 1e-5               #default = 1e-5
momentum = 0.9          #default = 0.9
weight_decay = 1e-4     #default = 1e-4
nesterov = False        #default = False
augmentation = True     #default = False

## Define Functions

### Loss Function

In [4]:
class BCEDiceLoss(nn.Module):
    def __init__(self):
        super().__init__()
    
    #Let x be the input and y be the target
    def forward(self, x, y):
        bce = F.binary_cross_entropy_with_logits(x, y)
        smooth = 1e-5
        x = torch.sigmoid(x)
        num = y.size(0)
        x = x.view(num, -1)
        y = y.view(num, -1)
        intersection = (x*y)
        dice = (2. * intersection.sum(1) + smooth) / (x.sum(1) + y.sum(1) + smooth)
        dice = 1 - dice.sum() / num

        return 0.5 * bce + dice

### Dataset

In [5]:
class LidcDataset(Dataset):
    def __init__(self, IMAGES_PATHS, MASK_PATHS, Albumentation=False):
        self.image_paths = IMAGES_PATHS
        self.mask_paths = MASK_PATHS
        self.albumentation = Albumentation

        self.albu_transformations = albu.Compose([
            albu.ElasticTransform(alpha=1.1,alpha_affine=0.5,sigma=5,p=0.15),
            albu.HorizontalFlip(p=0.5),
            ToTensorV2()
        ])
        self.transformations = transforms.Compose([transforms.ToTensor()])
    
    def transform(self, image, mask):
        #Transform to tensor
        if self.albumentation:
            #Make image and mask 3 dimensional
            image = image.reshape(512,512,1)
            mask = mask.reshape(512,512,1)
            #Convert datatype
            mask = mask.astype('uint8')
            #Apply augmentation
            augmented = self.albu_transformations(image=image,mask=mask)
            image = augmented['image']
            mask = augmented['mask']
            mask = mask.reshape([1,512,512])
        else:
            image = self.transformations(image)
            mask = self.transformations(mask)
        
        image, mask = image.type(torch.FloatTensor), mask.type(torch.FloatTensor)
        return image, mask

    def __getitem__(self, index):
        image = np.load(self.image_paths[index])
        mask = np.load(self.mask_paths[index])
        image, mask = self.transform(image, mask)
        return image, mask
    
    def __len__(self):
        return len(self.image_paths)

### Metrics

In [6]:
def iou_score(output, target):
    smooth = 1e-5
    if torch.is_tensor(output):
        output = torch.sigmoid(output).data.cpu().numpy()
    if torch.is_tensor(target):
        target = target.data.cpu().numpy()
    output_ = output > 0.5
    target_ = target > 0.5
    intersection = (output_ & target_).sum()
    union = (output_ | target_).sum()

    return (intersection + smooth) / (union + smooth)


def dice_coef(output, target):
    smooth = 1e-5
    #Sigmoid is used because the U-Net output is logit
    output = torch.sigmoid(output).view(-1).data.cpu().numpy()
    target = target.view(-1).data.cpu().numpy()
    intersection = (output*target).sum()

    return (2. * intersection + smooth) / \
        (output.sum() + target.sum() + smooth)


def sensitivity_metric(output, target):
    eps = 1e-5
    output = torch.sigmoid(output).view(-1).data.cpu()
    target = target.view(-1).data.cpu()
    # elements of confusion matrix
    tp = torch.sum(output * target) # True Positive
    fp = torch.sum(output * (1 - target)) # False Positive
    fn = torch.sum((1 - output) * target) # False Negative
    tn = torch.sum((1 - output) * (1 - target)) # True Negative
    # compute sensitivity
    sensitivity = (tp + eps) / (tp + fn + eps)
    
    return sensitivity.item()


def dice_coef2(output, target):
    "This metric is for validation"
    smooth = 1e-5
    output = output.view(-1)
    output = (output>0.5).float().cpu().numpy()
    target = target.view(-1).data.cpu().numpy()
    intersection = (output*target).sum()

    return (2. * intersection + smooth) / \
        (output.sum() + target.sum() + smooth)

### Utilities

In [7]:
def str_to_bool(v):
    if v.lower() in ['true', 1]:
        return True
    elif v.lower() in ['false', 0]:
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')

def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

class AverageMeter(object):
    #Computes and stores the average and current value
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
    
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

## Get Configuration

In [8]:
if augmentation == True:
    file_name = name + '_with_augmentation'
else:
    file_name = name + '_base'
os.makedirs('model_outputs/{}'.format(file_name), exist_ok=True)
print("Creating directory called ", file_name)

print('-' * 20)
print("Configuration Setting: ")
print("Model: ", name)
print("Max Epochs: ", epochs)
print("Batch Size: ", batch_size)
print("Number of Workers: ", num_workers)
print("Optimizer: ", optimizer)
print("Learning Rate: ", lr)
print("Augmentation: ", augmentation)

Creating directory called  UNet_with_augmentation
--------------------
Configuration Setting: 
Model:  UNet
Max Epochs:  100
Batch Size:  12
Number of Workers:  8
Optimizer:  Adam
Learning Rate:  1e-05
Augmentation:  True


## Create Model

In [9]:
criterion = BCEDiceLoss().cuda()
cudnn.benchmark = True

#Creating the model
print("Creating model...")
if name == 'NestedUNet':
    model = NestedUNet(num_classes=1)
else:
    model = UNet(n_channels=1, n_classes=1, bilinear=True)
model = model.cuda()

if torch.cuda.device_count() > 1:
    print("We can use ", torch.cuda.device_count(), " GPUs.")
    model = nn.DataParallel(model)

params = filter(lambda p: p.requires_grad, model.parameters())

if optimizer == 'Adam':
    optimizer = optim.Adam(params, lr=lr, weight_decay=weight_decay)
elif optimizer == 'SGD':
    optimizer = optim.SGD(params, lr=lr, momentum=momentum, nesterov=nesterov, weight_decay=weight_decay)
else:
    raise NotImplementedError
    
summary.summary(model,(1,512,512))

Creating model...


/scratch1/joseph.portugal/LIDC-IDRI-Segmentation-master/Unet/unet_parts.py:63: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
/scratch1/joseph.portugal/LIDC-IDRI-Segmentation-master/Unet/unet_parts.py:64: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  diffY // 2, diffY - diffY // 2])


Layer (type:depth-idx)                   Output Shape              Param #
├─DoubleConv: 1-1                        [-1, 64, 512, 512]        --
|    └─Sequential: 2-1                   [-1, 64, 512, 512]        --
|    |    └─Conv2d: 3-1                  [-1, 64, 512, 512]        640
|    |    └─BatchNorm2d: 3-2             [-1, 64, 512, 512]        128
|    |    └─ReLU: 3-3                    [-1, 64, 512, 512]        --
|    |    └─Conv2d: 3-4                  [-1, 64, 512, 512]        36,928
|    |    └─BatchNorm2d: 3-5             [-1, 64, 512, 512]        128
|    |    └─ReLU: 3-6                    [-1, 64, 512, 512]        --
├─Down: 1-2                              [-1, 128, 256, 256]       --
|    └─Sequential: 2-2                   [-1, 128, 256, 256]       --
|    |    └─MaxPool2d: 3-7               [-1, 64, 256, 256]        --
|    |    └─DoubleConv: 3-8              [-1, 128, 256, 256]       221,952
├─Down: 1-3                              [-1, 256, 128, 128]       --
|  

Layer (type:depth-idx)                   Output Shape              Param #
├─DoubleConv: 1-1                        [-1, 64, 512, 512]        --
|    └─Sequential: 2-1                   [-1, 64, 512, 512]        --
|    |    └─Conv2d: 3-1                  [-1, 64, 512, 512]        640
|    |    └─BatchNorm2d: 3-2             [-1, 64, 512, 512]        128
|    |    └─ReLU: 3-3                    [-1, 64, 512, 512]        --
|    |    └─Conv2d: 3-4                  [-1, 64, 512, 512]        36,928
|    |    └─BatchNorm2d: 3-5             [-1, 64, 512, 512]        128
|    |    └─ReLU: 3-6                    [-1, 64, 512, 512]        --
├─Down: 1-2                              [-1, 128, 256, 256]       --
|    └─Sequential: 2-2                   [-1, 128, 256, 256]       --
|    |    └─MaxPool2d: 3-7               [-1, 64, 256, 256]        --
|    |    └─DoubleConv: 3-8              [-1, 128, 256, 256]       221,952
├─Down: 1-3                              [-1, 256, 128, 128]       --
|  

## Load Dataset

In [10]:
#directory of Images and Masks folders (generated from preprocessing)                                         
IMAGE_DIR = '/scratch1/joseph.portugal/LIDC-IDRI Preprocessed/Image/'
MASK_DIR = '/scratch1/joseph.portugal/LIDC-IDRI Preprocessed/Mask/'                                                                 

#meta information
meta = pd.read_csv('/scratch1/joseph.portugal/LIDC-IDRI Preprocessed/Meta/meta.csv')     

#Get train/test label from metadata file
meta['original_image'] = meta['original_image'].apply(lambda x: IMAGE_DIR + "LIDC-IDRI-" + x[:4] + "/" + x + ".npy")
meta['mask_image'] = meta['mask_image'].apply(lambda x: MASK_DIR + "LIDC-IDRI-" + x[:4] + "/" + x + ".npy")


#Split into training and validation
train_meta = meta[meta['data_split']=='Train']
val_meta = meta[meta['data_split']=='Validation']

#Get training images into list
train_image_paths = list(train_meta['original_image'])
train_mask_paths = list(train_meta['mask_image'])

#Get validation images into list
val_image_paths = list(val_meta['original_image'])
val_mask_paths = list(val_meta['mask_image'])

print("*"*50)
print("Original images: {}, masks: {} for training.".format(len(train_image_paths),len(train_mask_paths)))
print("Original images: {}, masks: {} for validation.".format(len(val_image_paths),len(val_mask_paths)))
print("Ratio between Validation and Training is {:2f}".format(len(val_image_paths)/len(train_image_paths)))
print("*"*50)


#Creating custom LIDC dataset
train_dataset = LidcDataset(train_image_paths, train_mask_paths, augmentation)
val_dataset = LidcDataset(val_image_paths, val_mask_paths, augmentation)

#Creating Dataloader
train_loader = DataLoader(
  train_dataset,
  batch_size=batch_size,
  shuffle=True,
  pin_memory=True,
  drop_last=True,
  num_workers=6
)
val_loader = DataLoader(
  val_dataset,
  batch_size=batch_size,
  shuffle=False,
  pin_memory=True,
  drop_last=False,
  num_workers=6
)

**************************************************
Original images: 3587, masks: 3587 for training.
Original images: 1192, masks: 1192 for validation.
Ratio between Validation and Training is 0.332311
**************************************************


## Train the Model

In [11]:
log = pd.DataFrame(index=[], columns=['epoch','lr','loss','iou','dice','sensitivity','val_loss','val_iou'])

best_dice = 0
trigger = 0

for epoch in range(epochs):

    #Model Training
    avg_meters = {'loss': AverageMeter(), 'iou': AverageMeter(), 'dice': AverageMeter(), 'sensitivity': AverageMeter()}
    model.train()
    pbar = tqdm(total=len(train_loader)) #progress bar

    for i, data in enumerate(train_loader):

        input = data[0].cuda()
        target = data[1].cuda()
        output = model(input)

        #Get loss and metric
        loss = criterion(output, target)
        iou = iou_score(output, target)
        dice = dice_coef(output, target)
        sensitivity = sensitivity_metric(output, target)

        #Calculate the gradient and perform optimizing step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #Update average metrics
        avg_meters['loss'].update(loss.item(), input.size(0))
        avg_meters['iou'].update(iou, input.size(0))
        avg_meters['dice'].update(dice, input.size(0))
        avg_meters['sensitivity'].update(sensitivity, input.size(0))

        postfix = OrderedDict([
            ('loss', avg_meters['loss'].avg),
            ('iou', avg_meters['iou'].avg),
            ('dice', avg_meters['dice'].avg),
            ('sensitivity', avg_meters['sensitivity'].avg)
        ])
        pbar.set_postfix(postfix)
        pbar.update(1)
    pbar.close()

    train_log = OrderedDict([
        ('loss', avg_meters['loss'].avg),
        ('iou', avg_meters['iou'].avg),
        ('dice', avg_meters['dice'].avg),
        ('sensitivity', avg_meters['sensitivity'].avg)
    ])


    #Model Validation
    val_avg_meters = {'val_loss': AverageMeter(), 'val_iou': AverageMeter(), 'val_dice': AverageMeter(), 'val_sensitivity': AverageMeter()}
    model.eval()

    with torch.no_grad():
        val_pbar = tqdm(total=len(val_loader))
        for i, val_data in enumerate(val_loader):

            val_input = val_data[0].cuda()
            val_target = val_data[1].cuda()
            val_output = model(val_input)

            val_loss = criterion(val_output, val_target)
            val_iou = iou_score(val_output, val_target)
            val_dice = dice_coef(val_output, val_target)
            val_sensitivity = sensitivity_metric(val_output, val_target)

            val_avg_meters['val_loss'].update(val_loss.item(), val_input.size(0))
            val_avg_meters['val_iou'].update(val_iou, val_input.size(0))
            val_avg_meters['val_dice'].update(val_dice, val_input.size(0))
            val_avg_meters['val_sensitivity'].update(val_sensitivity, val_input.size(0))

            val_postfix = OrderedDict([
                ('val_loss', val_avg_meters['val_loss'].avg),
                ('val_iou', val_avg_meters['val_iou'].avg),
                ('val_dice', val_avg_meters['val_dice'].avg),
                ('val_sensitivity', val_avg_meters['val_sensitivity'].avg)
            ])
            val_pbar.set_postfix(val_postfix)
            val_pbar.update(1)
        val_pbar.close()

    val_log = OrderedDict([
        ('val_loss', val_avg_meters['val_loss'].avg),
        ('val_iou', val_avg_meters['val_iou'].avg),
        ('val_dice', val_avg_meters['val_dice'].avg),
        ('val_sensitivity', val_avg_meters['val_sensitivity'].avg)
    ])
    

    print('Training Epoch {}/{},  Training BCE Loss: {:.4f},  Training DICE: {:.4f},  Training IOU: {:.4f},  Training Sensitivity: {:.4f},  Validation BCE Loss: {:.4f},  Validation DICE: {:.4f},  Validation IOU: {:.4f},  Validation Sensitivity: {:.4f}'.format(
        epoch+1, epochs, train_log['loss'], train_log['dice'], train_log['iou'], train_log['sensitivity'], val_log['val_loss'], val_log['val_dice'], val_log['val_iou'], val_log['val_sensitivity']
    ))

    #Save values to csv file
    tmp = pd.Series([
        epoch,
        lr,
        train_log['loss'],
        train_log['iou'],
        train_log['dice'],
        val_log['val_loss'],
        val_log['val_iou'],
        val_log['val_dice'],
        val_log['val_sensitivity']
    ], index=['epoch', 'lr', 'loss', 'iou', 'dice', 'val_loss', 'val_iou', 'val_dice', 'val_sensitivity'])

    log = log.append(tmp, ignore_index=True)
    log.to_csv('model_outputs/{}/log.csv'.format(file_name), index=False)

    trigger += 1

    #If best DICE score, save the model
    if val_log['val_dice'] > best_dice:
        torch.save(model.state_dict(), 'model_outputs/{}/model.pth'.format(file_name))
        best_dice = val_log['val_dice']
        print("Saved new best model based on DICE metric!")
        trigger = 0
    
    if early_stopping >= 0 and trigger >= early_stopping:
        print("Early stopping.")
        break

    torch.cuda.empty_cache()

100%|██████████| 100/100 [00:50<00:00,  1.99it/s, val_loss=1.16, val_iou=0.00285, val_dice=0.00192, val_sensitivity=0.36]


Training Epoch 1/100,  Training BCE Loss: 1.1973,  Training DICE: 0.0016,  Training IOU: 0.0032,  Training Sensitivity: 0.4042,  Validation BCE Loss: 1.1550,  Validation DICE: 0.0019,  Validation IOU: 0.0029,  Validation Sensitivity: 0.3604
Saved new best model based on DICE metric!


100%|██████████| 100/100 [00:47<00:00,  2.09it/s, val_loss=1.14, val_iou=0.127, val_dice=0.00235, val_sensitivity=0.394] 


Training Epoch 2/100,  Training BCE Loss: 1.1471,  Training DICE: 0.0019,  Training IOU: 0.0383,  Training Sensitivity: 0.3894,  Validation BCE Loss: 1.1418,  Validation DICE: 0.0024,  Validation IOU: 0.1268,  Validation Sensitivity: 0.3936
Saved new best model based on DICE metric!


100%|██████████| 100/100 [00:48<00:00,  2.08it/s, val_loss=1.13, val_iou=0.208, val_dice=0.00272, val_sensitivity=0.425] 


...

Training Epoch 98/100,  Training BCE Loss: 0.1733,  Training DICE: 0.8901,  Training IOU: 0.8356,  Training Sensitivity: 0.9287,  Validation BCE Loss: 0.5590,  Validation DICE: 0.5594,  Validation IOU: 0.4592,  Validation Sensitivity: 0.5241


100%|██████████| 100/100 [00:48<00:00,  2.06it/s, val_loss=0.575, val_iou=0.422, val_dice=0.53, val_sensitivity=0.487]


Training Epoch 99/100,  Training BCE Loss: 0.1742,  Training DICE: 0.8872,  Training IOU: 0.8278,  Training Sensitivity: 0.9251,  Validation BCE Loss: 0.5746,  Validation DICE: 0.5298,  Validation IOU: 0.4218,  Validation Sensitivity: 0.4871


100%|██████████| 100/100 [00:48<00:00,  2.08it/s, val_loss=0.619, val_iou=0.403, val_dice=0.507, val_sensitivity=0.45]

Training Epoch 100/100,  Training BCE Loss: 0.1675,  Training DICE: 0.8905,  Training IOU: 0.8316,  Training Sensitivity: 0.9269,  Validation BCE Loss: 0.6187,  Validation DICE: 0.5072,  Validation IOU: 0.4033,  Validation Sensitivity: 0.4504
